## Setup

In [1]:
%run ../setup_llm.py


🚀 LLM Setup Utility Test

🔍 Checking LLM Configuration...
📡 Provider: OpenAI (Direct)
✅ OPENAI_API_KEY is set

📋 Configuration:
   OPENAI_MODEL_NAME: gpt-4o

✅ OpenAI setup verified successfully!

📝 Testing model initialization...
✅ Model created: gpt-4o via OpenAI
✅ Test response: Hello!


In [2]:
import sys
sys.path.insert(0, '..')  # Add parent directory to path for setup_llm

from setup_llm import get_chat_model  # Use LLM helper
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.agents import create_agent  # LangChain v1.0 - new standard for building agents
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver  # Recommended checkpointer for development/testing
from typing import TypedDict, Annotated, List
import operator
import os
from datetime import datetime

# Initialize model
# Initialize model using DIAL (Azure OpenAI)
model = get_chat_model(temperature=0)

print("✅ Environment ready")


✅ Environment ready


---

## Part 0: LangChain vs LangGraph – Understanding the Framework Strategy

### The Modern AI Framework Stack

With the release of LangChain v1.0, there's a clear division of responsibilities between LangChain and LangGraph:

```
┌─────────────────────────────────────────────────────────────────────┐
│                        YOUR APPLICATION                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│   ┌───────────────────────────────────────────────────────────────┐ │
│   │                    LangChain (High Level)                     │ │
│   │                                                               │ │
│   │  • create_agent() - Pre-built agent architecture              │ │
│   │  • Middleware system - Composable behaviors                   │ │
│   │  • Quick prototyping - Build agents in <10 lines              │ │
│   │  • Common patterns - Built-in HITL, memory, tools             │ │
│   └───────────────────────────────────────────────────────────────┘ │
│                              ▲                                       │
│                              │ Built ON TOP of                       │
│                              ▼                                       │
│   ┌───────────────────────────────────────────────────────────────┐ │
│   │                   LangGraph (Foundation)                      │ │
│   │                                                               │ │
│   │  • StateGraph - Custom graph orchestration                    │ │
│   │  • Nodes & Edges - Fine-grained control flow                  │ │
│   │  • Checkpointing - Persistence & time travel                  │ │
│   │  • Send API - Dynamic parallel execution                      │ │
│   └───────────────────────────────────────────────────────────────┘ │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### When to Use Each

| Scenario | Use LangChain | Use LangGraph |
|----------|---------------|---------------|
| **Quick prototype** | ✅ `create_agent()` | ❌ Overkill |
| **Simple chatbot with tools** | ✅ Middleware handles complexity | ❌ |
| **Custom multi-step workflow** | ❌ | ✅ `StateGraph` |
| **Complex branching logic** | ❌ | ✅ Conditional edges |
| **Map-reduce parallelization** | ❌ | ✅ `Send` API |
| **Production with customization** | Start with LangChain, drop to LangGraph when needed |

### The Key Insight

> **LangChain's `create_agent` runs on LangGraph under the hood.**
> 
> Start with LangChain for simplicity, and drop down to LangGraph's graph APIs when you need more control.

### Framework Responsibilities

| Framework | Primary Responsibility |
|-----------|----------------------|
| **LangChain** | Agent creation, middleware, tool binding, prompts |
| **LangGraph** | Orchestration, state management, persistence, execution flow |
| **langchain-core** | Base abstractions (messages, tools, models, runnables) |

In [3]:
# Example: Same agent, two approaches

# ════════════════════════════════════════════════════════════════════
# APPROACH 1: LangChain create_agent (High-level, Quick)
# ════════════════════════════════════════════════════════════════════

from langchain.agents import create_agent
from langchain_core.tools import tool

@tool
def get_weather(city: str) -> str:
    """Get weather for a city."""
    return f"Weather in {city}: 22°C, sunny"

# Just 3 lines to create a working agent!
simple_agent = create_agent(
    model=model,
    tools=[get_weather]
)

print("✅ LangChain Approach: Agent created in 3 lines")
print("   Best for: Quick prototypes, simple use cases")

result = simple_agent.invoke({"messages": [("user", "What's the weather in Paris?")]})
print(f"   Result: {result['messages'][-1].content[:100]}...")

✅ LangChain Approach: Agent created in 3 lines
   Best for: Quick prototypes, simple use cases
   Result: The weather in Paris is currently 22°C and sunny....


In [4]:
# ════════════════════════════════════════════════════════════════════
# APPROACH 2: LangGraph StateGraph (Low-level, Full Control)
# ════════════════════════════════════════════════════════════════════

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from typing import TypedDict, Annotated, Literal
import operator

# Define state explicitly
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]

# Bind tools to model
model_with_tools = model.bind_tools([get_weather])

# Define nodes
def call_model(state: AgentState):
    response = model_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: AgentState) -> Literal["tools", "end"]:
    if state["messages"][-1].tool_calls:
        return "tools"
    return "end"

# Build graph manually
builder = StateGraph(AgentState)
builder.add_node("agent", call_model)
builder.add_node("tools", ToolNode([get_weather]))
builder.add_edge(START, "agent")
builder.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
builder.add_edge("tools", "agent")

custom_agent = builder.compile()

print("\n✅ LangGraph Approach: Full control over graph structure")
print("   Best for: Custom workflows, complex routing, max flexibility")
print("   Lines of code: ~25 vs ~3 for LangChain")

# Same result, more control
result = custom_agent.invoke({"messages": [("user", "What's the weather in Tokyo?")]})
print(f"   Result: {result['messages'][-1].content[:100]}...")


✅ LangGraph Approach: Full control over graph structure
   Best for: Custom workflows, complex routing, max flexibility
   Lines of code: ~25 vs ~3 for LangChain
   Result: The weather in Tokyo is currently 22°C and sunny....


---

## Part 1: Advanced Tools with Pydantic

### Why Pydantic?

**Pydantic** provides:
- Type-safe data validation
- Automatic JSON schema generation
- Forces LLM to return structured data

### Use Cases
- Planning agents (structured task lists)
- Data extraction (structured info from text)
- Form filling (validated user inputs)
- API responses (typed outputs)

In [5]:
from pydantic import BaseModel, Field
from typing import List, Optional
from enum import Enum

# Example 1: Simple Pydantic model
class TravelPreferences(BaseModel):
    """User's travel preferences."""
    destination: str = Field(description="The destination city")
    budget: str = Field(description="Budget level: low, medium, or high")
    interests: List[str] = Field(description="List of interests")
    duration_days: int = Field(description="Trip duration in days")

# Create a structured output model
travel_extractor = model.with_structured_output(TravelPreferences, method="function_calling")

# Test it
result = travel_extractor.invoke([
    HumanMessage(content="I want to visit Tokyo for a week. I love sushi and anime, and I have a moderate budget.")
])

print("📋 Extracted Travel Preferences:")
print(f"   Destination: {result.destination}")
print(f"   Budget: {result.budget}")
print(f"   Interests: {result.interests}")
print(f"   Duration: {result.duration_days} days")
print(f"\n   JSON: {result.model_dump_json(indent=2)}")

📋 Extracted Travel Preferences:
   Destination: Tokyo
   Budget: medium
   Interests: ['sushi', 'anime']
   Duration: 7 days

   JSON: {
  "destination": "Tokyo",
  "budget": "medium",
  "interests": [
    "sushi",
    "anime"
  ],
  "duration_days": 7
}


### Structured Output Methods Comparison

LangChain offers multiple ways to get structured output:

| Method | Description | Best For |
|--------|-------------|----------|
| `method="json_schema"` | Native structured output (preferred) | Most modern models |
| `method="function_calling"` | Uses tool/function call under the hood | Compatibility |
| `include_raw=True` | Returns both raw AIMessage and parsed output | Debugging |
| `response_format` in `create_agent` | Direct schema on agent | Agents with structured final response |

In [6]:
# Modern structured output patterns

# Pattern 1: json_schema method (preferred for modern models)
class MovieReview(BaseModel):
    """Structured movie review."""
    title: str = Field(description="Movie title")
    rating: float = Field(description="Rating from 1-10", ge=1, le=10)
    pros: List[str] = Field(description="Positive aspects")
    cons: List[str] = Field(description="Negative aspects")
    recommendation: bool = Field(description="Would you recommend?")

# Use json_schema for native structured output
reviewer_json_schema = model.with_structured_output(MovieReview, method="json_schema")

print("Pattern 1: json_schema (native structured output)")
print("-" * 50)
review = reviewer_json_schema.invoke([
    HumanMessage(content="Review the movie 'Inception' by Christopher Nolan")
])
print(review)
print(f"Title: {review.title}")
print(f"Rating: {review.rating}/10")
print(f"Pros: {', '.join(review.pros[:2])}...")
print(f"Recommend: {'Yes' if review.recommendation else 'No'}")

Pattern 1: json_schema (native structured output)
--------------------------------------------------
title='Inception' rating=9.0 pros=['Innovative and complex storyline', 'Stunning visual effects', 'Strong performances by the cast, especially Leonardo DiCaprio', 'Thought-provoking themes about dreams and reality', 'Excellent direction by Christopher Nolan', 'Memorable and intense musical score by Hans Zimmer'] cons=['Complex plot may be confusing for some viewers', 'Requires multiple viewings to fully understand'] recommendation=True
Title: Inception
Rating: 9.0/10
Pros: Innovative and complex storyline, Stunning visual effects...
Recommend: Yes


In [7]:
# Pattern 2: include_raw=True for debugging
reviewer_with_raw = model.with_structured_output(MovieReview, include_raw=True)

print("\nPattern 2: include_raw=True (for debugging)")
print("-" * 50)
result_with_raw = reviewer_with_raw.invoke([
    HumanMessage(content="Quick review of 'The Matrix'")
])

print(f"Raw AIMessage type: {type(result_with_raw['raw']).__name__}")
print(f"Raw content preview: {str(result_with_raw['raw'].content)[:100]}...")
print(f"Parsed result: {result_with_raw['parsed'].title} - {result_with_raw['parsed'].rating}/10")
print("\n✅ Use include_raw=True when debugging structured output issues")


Pattern 2: include_raw=True (for debugging)
--------------------------------------------------
Raw AIMessage type: AIMessage
Raw content preview: {"title":"The Matrix","rating":9,"pros":["Innovative special effects and groundbreaking visual style...
Parsed result: The Matrix - 9.0/10

✅ Use include_raw=True when debugging structured output issues


In [8]:
# Pattern 3: response_format with create_agent
# Forces the agent's FINAL response to be structured

class TaskResult(BaseModel):
    """Structured task completion report."""
    task_completed: bool = Field(description="Whether the task was completed")
    summary: str = Field(description="Brief summary of what was done")
    steps_taken: List[str] = Field(description="Steps the agent took")
    next_actions: Optional[List[str]] = Field(description="Suggested follow-up actions", default=None)

@tool
def search_web(query: str) -> str:
    """Search the web for information."""
    return f"Search results for '{query}': Found 5 relevant articles about the topic."

@tool
def save_note(content: str) -> str:
    """Save a note for later reference."""
    return f"Note saved: {content[:50]}..."

# Agent with structured final response
structured_agent = create_agent(
    model=model,
    tools=[search_web, save_note],
    response_format=TaskResult  # Agent's final output will be structured!
)

print("Pattern 3: response_format with create_agent")
print("-" * 50)
print("Agent will return structured TaskResult after completing the task")

result = structured_agent.invoke({
    "messages": [("user", "Search for information about LangGraph and save a note about it")]
})

# The final response is structured
final_msg = result["messages"][-1]
print(f"\n📋 Structured Response:")
print(f"   Task Completed: {final_msg.content}")
print("\n✅ response_format ensures consistent, parseable agent outputs")

Pattern 3: response_format with create_agent
--------------------------------------------------
Agent will return structured TaskResult after completing the task

📋 Structured Response:
   Task Completed: {"task_completed":true,"summary":"Searched for information about LangGraph and saved a note summarizing its relevance to language processing or graph-based data structures.","steps_taken":["Searched the web for information about LangGraph.","Saved a note summarizing the information found about LangGraph."],"next_actions":null}

✅ response_format ensures consistent, parseable agent outputs


### Example: Complex Structured Output for Planning

In [9]:
class PriorityLevel(str, Enum):
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"

class AgentType(str, Enum):
    FLIGHT_BOOKING = "flight_booking"
    HOTEL_BOOKING = "hotel_booking"
    ACTIVITIES = "activities"
    GENERAL = "general"

class Subtask(BaseModel):
    """A single subtask in a plan."""
    task_id: int = Field(description="Unique task identifier")
    description: str = Field(description="What needs to be done")
    assigned_agent: AgentType = Field(description="Which agent handles this")
    priority: PriorityLevel = Field(description="Task priority")
    estimated_minutes: int = Field(description="Estimated completion time")

class TravelPlan(BaseModel):
    """Complete travel plan with subtasks."""
    goal: str = Field(description="The overall travel goal")
    destination: str = Field(description="Main destination")
    subtasks: List[Subtask] = Field(description="Ordered list of subtasks")
    total_estimated_cost: str = Field(description="Budget estimate")

# Create planner
planner = model.with_structured_output(TravelPlan, method="function_calling")

plan = planner.invoke([
    SystemMessage(content="""You are a travel planning agent. Create detailed plans with:
- flight_booking: For flights
- hotel_booking: For accommodations
- activities: For tours and attractions
- general: For other tasks"""),
    HumanMessage(content="Plan a 5-day family trip from Singapore to Melbourne with 2 kids. Budget around $3000. We love museums and outdoor activities.")
])

print("📋 TRAVEL PLAN")
print("=" * 50)
print(f"Goal: {plan.goal}")
print(f"Destination: {plan.destination}")
print(f"Estimated Cost: {plan.total_estimated_cost}")
print(f"\nSubtasks ({len(plan.subtasks)}):")

for task in plan.subtasks:
    print(f"\n  {task.task_id}. [{task.assigned_agent.value.upper()}] ({task.priority.value} priority)")
    print(f"     {task.description}")
    print(f"     ⏱️ Est. {task.estimated_minutes} min")

📋 TRAVEL PLAN
Goal: 5-day family trip from Singapore to Melbourne focusing on museums and outdoor activities.
Destination: Melbourne
Estimated Cost: $3000

Subtasks (8):

  1. [FLIGHT_BOOKING] (high priority)
     Book round-trip flights from Singapore to Melbourne for 2 adults and 2 children.
     ⏱️ Est. 60 min

  2. [HOTEL_BOOKING] (high priority)
     Reserve a family-friendly hotel in Melbourne with easy access to public transport.
     ⏱️ Est. 45 min

  3. [ACTIVITIES] (medium priority)
     Plan a visit to the Melbourne Museum and Scienceworks for family-friendly educational experiences.
     ⏱️ Est. 30 min

  4. [ACTIVITIES] (medium priority)
     Organize a day trip to the Great Ocean Road for scenic views and outdoor activities.
     ⏱️ Est. 40 min

  5. [ACTIVITIES] (low priority)
     Schedule a visit to the Royal Botanic Gardens for a family picnic and exploration.
     ⏱️ Est. 20 min

  6. [ACTIVITIES] (low priority)
     Book tickets for a family-friendly show or event a

---

## Part 2: Memory & Persistence

### Why Memory Matters

Without memory, agents forget everything between turns:

```
WITHOUT MEMORY
══════════════════════════════════════
User: "My name is Alice"
Agent: "Nice to meet you, Alice!"

User: "What's my name?"
Agent: "I don't know your name." ❌


WITH MEMORY
══════════════════════════════════════
User: "My name is Alice"
Agent: "Nice to meet you, Alice!"

User: "What's my name?"
Agent: "Your name is Alice!" ✅
```

### Setting Up Memory with MemorySaver

In [10]:
from langgraph.checkpoint.memory import InMemorySaver

# Create memory store
# InMemorySaver is the recommended checkpointer for development/testing
memory = InMemorySaver()

# Simple tool
@tool
def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Create agent WITH memory using create_agent (LangChain v1.0)
agent_with_memory = create_agent(
    model=model,
    tools=[get_current_time],
    checkpointer=memory  # This enables memory!
)

print("✅ Agent with memory created")
print("   Checkpointer: InMemorySaver (in-memory)")
print("   Memory type: Thread-based (per conversation)")

✅ Agent with memory created
   Checkpointer: InMemorySaver (in-memory)
   Memory type: Thread-based (per conversation)


### Demo: Multi-Turn Conversation

In [11]:
# Thread ID = conversation identifier
thread_id = "user_alice_123"
config = {"configurable": {"thread_id": thread_id}}

print(f"🧵 Starting conversation in thread: {thread_id}\n")

# Turn 1: User introduces themselves
print("👤 User: Hi! My name is Alice and I love traveling to Japan.\n")

response1 = agent_with_memory.invoke(
    {"messages": [("user", "Hi! My name is Alice and I love traveling to Japan.")]},
    config=config
)

print(f"🤖 Agent: {response1['messages'][-1].content}\n")
print("-" * 60)

🧵 Starting conversation in thread: user_alice_123

👤 User: Hi! My name is Alice and I love traveling to Japan.

🤖 Agent: Hi Alice! That sounds amazing! Japan is such a beautiful country with a rich culture and so many incredible places to explore. Do you have a favorite spot in Japan, or is there somewhere specific you'd like to visit next?

------------------------------------------------------------


In [12]:
# Turn 2: Agent should remember name
print("\n👤 User: What's my name?\n")

response2 = agent_with_memory.invoke(
    {"messages": [("user", "What's my name?")]},
    config=config  # Same thread!
)

print(f"🤖 Agent: {response2['messages'][-1].content}\n")
print("-" * 60)


👤 User: What's my name?

🤖 Agent: Your name is Alice.

------------------------------------------------------------


In [13]:
# Turn 3: Agent should remember preference
print("\n👤 User: What do I like?\n")

response3 = agent_with_memory.invoke(
    {"messages": [("user", "What do I like?")]},
    config=config
)

print(f"🤖 Agent: {response3['messages'][-1].content}")
print("\n✅ Agent remembers both name AND preference!")


👤 User: What do I like?

🤖 Agent: You love traveling to Japan.

✅ Agent remembers both name AND preference!


### Thread Isolation: Multiple Users

In [14]:
# Different thread = different conversation
config_user2 = {"configurable": {"thread_id": "user_bob_456"}}

print("🧵 New thread: user_bob_456\n")
print("👤 User 2: I'm Bob, I'm 30 years old.\n")

response_bob = agent_with_memory.invoke(
    {"messages": [("user", "I'm Bob, I'm 30 years old.")]},
    config=config_user2
)

print(f"🤖 Agent: {response_bob['messages'][-1].content}")

🧵 New thread: user_bob_456

👤 User 2: I'm Bob, I'm 30 years old.

🤖 Agent: Hello Bob! How can I assist you today?


In [15]:
# Go back to Alice's thread
print("\n" + "=" * 60)
print("🧵 Back to Alice's thread\n")
print("👤 User (Alice): How old is Bob?\n")

response_alice = agent_with_memory.invoke(
    {"messages": [("user", "How old is Bob?")]},
    config=config  # Alice's thread
)

print(f"🤖 Agent: {response_alice['messages'][-1].content}")
print("\n✅ Threads are isolated! Alice doesn't know about Bob.")


🧵 Back to Alice's thread

👤 User (Alice): How old is Bob?

🤖 Agent: I'm sorry, but I don't have any information about Bob's age. Could you provide more context or details?

✅ Threads are isolated! Alice doesn't know about Bob.


---

## Part 3: Human-in-the-Loop (HITL)

### Why HITL?

Some actions need human oversight:
- **Expensive operations** (booking flights, making purchases)
- **Sensitive actions** (deleting data, sending emails)
- **Quality control** (reviewing agent outputs)
- **Error correction** (fixing agent mistakes)

```
HITL PATTERN
══════════════════════════════════════════════════════════════

    Agent → Plans action → PAUSE → Human reviews → Resume
```

In [16]:
from langchain.agents.middleware import HumanInTheLoopMiddleware

# Simple tool that the HITL middleware will gate
@tool
def book_expensive_flight(
    origin: Annotated[str, "Departure city"],
    destination: Annotated[str, "Arrival city"],
    date: Annotated[str, "Travel date (YYYY-MM-DD)"],
    passengers: Annotated[int, "Number of passengers"]
) -> str:
    """
    Book a flight. This is an expensive action that requires human approval.
    """
    estimated_cost = passengers * 500
    return f"✅ Flight booked: {origin} → {destination} on {date} for {passengers} pax. Cost: ${estimated_cost}. Confirmation: FL{hash(f'{origin}{destination}{date}') % 10000}"

# Create HITL-enabled agent using middleware approach (LangChain v1.0)
hitl_memory = InMemorySaver()

hitl_agent = create_agent(
    model=model,
    tools=[book_expensive_flight],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"book_expensive_flight": True},  # Require approval for this tool
            description_prefix="Flight booking pending approval"
        )
    ],
    checkpointer=hitl_memory
)

print("✅ HITL agent created with middleware")
print("   Tool: book_expensive_flight (requires approval via middleware)")

✅ HITL agent created with middleware
   Tool: book_expensive_flight (requires approval via middleware)


### Demo: Interrupt and Resume Workflow

In [17]:
# Step 1: User requests booking
booking_thread = "booking_flow_1"
booking_config = {"configurable": {"thread_id": booking_thread}}

user_query = "Book a flight from New York to London on 2025-06-15 for 2 passengers"

print(f"👤 User: {user_query}\n")
print("🤖 Agent processing...\n")

# This will interrupt when the tool is called (middleware pattern)
response = hitl_agent.invoke(
    {"messages": [("user", user_query)]},
    config=booking_config
)

# Check if we hit an interrupt
if "__interrupt__" in response:
    print("⏸️  Agent paused (waiting for approval)")
    interrupts = response["__interrupt__"]
    for interrupt_ in interrupts:
        print(f"\n📋 Pending action details:")
        # Just print the value which contains the action info
        print(f"   {interrupt_.value}")
else:
    print("✅ Completed without interrupt")
    print(f"Result: {response['messages'][-1].content}")

👤 User: Book a flight from New York to London on 2025-06-15 for 2 passengers

🤖 Agent processing...

⏸️  Agent paused (waiting for approval)

📋 Pending action details:
   {'action_requests': [{'name': 'book_expensive_flight', 'args': {'origin': 'New York', 'destination': 'London', 'date': '2025-06-15', 'passengers': 2}, 'description': "Flight booking pending approval\n\nTool: book_expensive_flight\nArgs: {'origin': 'New York', 'destination': 'London', 'date': '2025-06-15', 'passengers': 2}"}], 'review_configs': [{'action_name': 'book_expensive_flight', 'allowed_decisions': ['approve', 'edit', 'reject']}]}


In [18]:
# Step 2: Check what the agent is waiting for
state = hitl_agent.get_state(booking_config)

print("🔍 Current State:")
print(f"   Next step: {state.next}")

if state.tasks:
    task = state.tasks[0]
    if hasattr(task, 'interrupts') and task.interrupts:
        print(f"\n🛑 Waiting for approval:")
        # The value is now in action_requests format
        value = task.interrupts[0].value
        if 'action_requests' in value:
            for req in value['action_requests']:
                print(f"   Tool: {req['name']}")
                print(f"   Args: {req['args']}")

🔍 Current State:
   Next step: ('HumanInTheLoopMiddleware.after_model',)

🛑 Waiting for approval:
   Tool: book_expensive_flight
   Args: {'origin': 'New York', 'destination': 'London', 'date': '2025-06-15', 'passengers': 2}


In [19]:
from langgraph.types import Command

# Step 2: Human approves
print("✅ Human approves the action\n")

# Resume with approval
response = hitl_agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=booking_config  # Same thread to resume
)

print("🎉 FINAL RESULT:")
print(f"   {response['messages'][-1].content}")

✅ Human approves the action

🎉 FINAL RESULT:
   The flight from New York to London on 2025-06-15 for 2 passengers has been successfully booked. The total cost is $1000, and your confirmation number is FL4467.


### Demo: Rejection Scenario

In [20]:
# Demonstrate rejection flow with a NEW agent (to avoid state conflicts)
# Create a completely fresh agent for the rejection demo

from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware

# Create a fresh checkpointer and agent for rejection demo
rejection_checkpointer = InMemorySaver()
rejection_agent = create_agent(
    model,
    [book_expensive_flight],
    checkpointer=rejection_checkpointer,
    middleware=[HumanInTheLoopMiddleware(interrupt_on={"book_expensive_flight": True})]
)

# New thread for rejection scenario
reject_thread = "rejection_demo_thread"
reject_config = {"configurable": {"thread_id": reject_thread}}

print("👤 User: Book a flight from Tokyo to Sydney on 2025-07-01 for 5 passengers\n")

# Start the booking request (will pause for approval)
rejection_agent.invoke(
    {"messages": [("user", "Book a flight from Tokyo to Sydney on 2025-07-01 for 5 passengers")]},
    config=reject_config
)

# Check pending action and display info
state = rejection_agent.get_state(reject_config)
if state.tasks and state.tasks[0].interrupts:
    value = state.tasks[0].interrupts[0].value
    action_requests = value.get("action_requests", [])
    if action_requests:
        args = action_requests[0].get("args", {})
        print(f"📋 Pending booking request:")
        print(f"   Route: {args.get('origin')} → {args.get('destination')}")
        print(f"   Date: {args.get('date')}")
        print(f"   Passengers: {args.get('passengers')}\n")
    
    print("👨‍💼 Human Decision: REJECTED (too expensive for 5 passengers!)\n")
    
    # Resume with rejection using proper Command format
    response = rejection_agent.invoke(
        Command(resume={"decisions": [{"type": "reject"}]}),
        config=reject_config
    )
    
    print(f"🤖 Agent Response: {response['messages'][-1].content}")

👤 User: Book a flight from Tokyo to Sydney on 2025-07-01 for 5 passengers

📋 Pending booking request:
   Route: Tokyo → Sydney
   Date: 2025-07-01
   Passengers: 5

👨‍💼 Human Decision: REJECTED (too expensive for 5 passengers!)

🤖 Agent Response: It seems that the flight booking request was not approved. If you have any other requests or need further assistance, please let me know!


---

## Part 4: Time Travel Debugging

### What is Time Travel?

Memory/checkpointing enables **time travel** - you can:
- View all conversation checkpoints
- Go back to any previous state
- "Replay" from a different point
- Fix agent mistakes by rolling back

In [21]:
# Get conversation history for Alice's thread
history = agent_with_memory.get_state_history(config)

print("📜 Conversation Checkpoints:")
print("=" * 50)

checkpoints = []
for i, state in enumerate(history, 1):
    checkpoints.append(state)
    checkpoint_id = state.config['configurable'].get('checkpoint_id', 'N/A')[:20]
    msg_count = len(state.values.get('messages', []))
    
    print(f"\n{i}. Checkpoint: {checkpoint_id}...")
    print(f"   Messages: {msg_count}")
    print(f"   Next: {state.next}")
    
    if i >= 5:  # Show first 5
        remaining = len(list(history)) - 5
        if remaining > 0:
            print(f"\n   ... and {remaining} more checkpoints")
        break

print(f"\n📊 Total checkpoints: {len(checkpoints)}")

📜 Conversation Checkpoints:

1. Checkpoint: 1f0d03f2-1a0f-68c0-8...
   Messages: 8
   Next: ()

2. Checkpoint: 1f0d03f2-1316-6f64-8...
   Messages: 7
   Next: ('model',)

3. Checkpoint: 1f0d03f2-1315-6e20-8...
   Messages: 6
   Next: ('__start__',)

4. Checkpoint: 1f0d03f1-bf20-6888-8...
   Messages: 6
   Next: ()

5. Checkpoint: 1f0d03f1-b90e-6756-8...
   Messages: 5
   Next: ('model',)

   ... and 2 more checkpoints

📊 Total checkpoints: 5


In [22]:
# Replay from an earlier checkpoint
if len(checkpoints) >= 2:
    older_checkpoint = checkpoints[-1]  # Get an older checkpoint
    
    print("⏪ Time Travel Demo")
    print("=" * 50)
    
    # Show current state
    current_state = agent_with_memory.get_state(config)
    print(f"\nCurrent state: {len(current_state.values.get('messages', []))} messages")
    
    # Create config pointing to old checkpoint
    old_checkpoint_id = older_checkpoint.config['configurable'].get('checkpoint_id')
    if old_checkpoint_id:
        replay_config = {
            "configurable": {
                "thread_id": thread_id,
                "checkpoint_id": old_checkpoint_id
            }
        }
        
        old_state = agent_with_memory.get_state(replay_config)
        print(f"Restored state: {len(old_state.values.get('messages', []))} messages")
        print("\n🎯 Use Case: If agent made a mistake, you can rollback and retry!")

⏪ Time Travel Demo

Current state: 8 messages
Restored state: 5 messages

🎯 Use Case: If agent made a mistake, you can rollback and retry!


---

## Part 5: Putting It All Together

Let's build a complete agent that combines:
- Multiple tools
- Structured outputs
- Memory
- Conditional HITL approval

In [23]:
# Tools with different approval thresholds
@tool
def search_flights(origin: str, destination: str, date: str) -> str:
    """Search for available flights (no approval needed)."""
    return f"Found 3 flights {origin} → {destination} on {date}: $350, $420, $500"

@tool
def book_flight_smart(
    flight_id: str,
    price: Annotated[float, "Flight price in USD"],
    passengers: int
) -> str:
    """
    Book a flight. Requires approval if total cost > $1000.
    """
    total_cost = price * passengers
    
    if total_cost > 1000:
        # High-value booking needs approval
        approval = interrupt(
            value={"flight_id": flight_id, "total_cost": total_cost},
            message=f"🛑 High-value booking: ${total_cost}"
        )
        
        if approval != "APPROVED":
            return "❌ Booking cancelled"
    
    return f"✅ Booked {flight_id} for {passengers} passengers. Total: ${total_cost}"

@tool
def get_user_profile() -> str:
    """Get the current user's profile and preferences."""
    return "User: Alice | Preferences: Window seat, Vegetarian meal | Status: Gold member"

print("✅ Smart booking tools defined")

✅ Smart booking tools defined


In [24]:
# Create the complete agent using create_agent (LangChain v1.0)
complete_memory = InMemorySaver()

complete_agent = create_agent(
    model=model,
    tools=[search_flights, book_flight_smart, get_user_profile],
    checkpointer=complete_memory,
    system_prompt="""You are a helpful travel assistant. You have access to:
- search_flights: Find available flights
- book_flight_smart: Book flights (needs approval for expensive bookings)
- get_user_profile: Get user preferences

Always search before booking. Consider user preferences."""
)

print("✅ Complete travel agent ready")

✅ Complete travel agent ready


In [25]:
# Test complete flow
travel_thread = "travel_assistant_1"
travel_config = {"configurable": {"thread_id": travel_thread}}

# Small booking (no approval needed)
print("📝 TEST 1: Small Booking (under $1000)")
print("=" * 50)

response = complete_agent.invoke(
    {"messages": [("user", "Book the cheapest flight from NYC to Boston tomorrow for 1 person")]},
    config=travel_config
)

print(f"\n🤖 Agent: {response['messages'][-1].content}")

📝 TEST 1: Small Booking (under $1000)

🤖 Agent: I have successfully booked the cheapest flight from NYC to Boston for tomorrow at a total cost of $350. Your preferences for a window seat and vegetarian meal have been noted. Safe travels, Alice!


In [26]:
from langgraph.types import Command

# Step 2: Human approves
print("✅ Human approves the action\n")

# Resume with approval
response = hitl_agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=booking_config  # Same thread to resume
)

print("🎉 FINAL RESULT:")
print(f"   {response['messages'][-1].content}")

✅ Human approves the action

🎉 FINAL RESULT:
   The flight from New York to London on 2025-06-15 for 2 passengers has been successfully booked. The total cost is $1000, and your confirmation number is FL4467.


---

## 🎯 Key Takeaways

### Advanced Tools with Pydantic

```python
# Structured output
class Output(BaseModel):
    field: type = Field(description="...")

extractor = model.with_structured_output(Output, method="function_calling")

# Annotated parameters
@tool
def func(param: Annotated[str, "description"]) -> str:
    ...
```

### Memory & Persistence

| Pattern | Use Case | Implementation |
|---------|----------|---------------|
| **Thread-based** | Per-user conversations | `thread_id` in config |
| **Checkpointing** | Save/restore state | InMemorySaver (dev) / PostgresSaver (prod) |
| **Time travel** | Error recovery | Restore old checkpoint |

### Human-in-the-Loop

| Pattern | Use Case | Implementation |
|---------|----------|---------------|
| **Approval gate** | Expensive actions | `interrupt()` in tool |
| **Threshold-based** | Conditional approval | Check value, then interrupt |
| **Resume** | Continue after approval | `Command(resume=value)` |

---

## 🎉 Congratulations!

You've completed **Session 1** of the Agentic AI Workshop!

### What You Learned

| Module | Topics |
|--------|--------|
| **Module 0** | What are agents, why they matter, first agent |
| **Module 1** | ReAct framework, LangGraph foundations, state |
| **Module 2** | Workflow patterns: chaining, routing, parallelization |
| **Module 3** | Multi-agent systems: supervisor, handoffs, planning |
| **Module 4** | Framework comparison and selection |
| **Module 5** | Advanced tools, memory, HITL, time travel |

### What's Next (Session 2)

- **Production-grade persistence** (PostgreSQL, SQLite)
- **Advanced architectural patterns**
- **Deep agents & reflection**
- **Context engineering**

---

## Additional Resources

- [LangGraph Documentation](https://docs.langchain.com/oss/python/langgraph/overview)
- [LangChain v1 Release Notes](https://docs.langchain.com/oss/python/releases/langchain-v1)
- [Persistence & Checkpointing](https://docs.langchain.com/oss/python/langgraph/persistence)
- [Human-in-the-Loop Patterns](https://docs.langchain.com/oss/python/langgraph/interrupts)
- [LangGraph v1 Migration Guide](https://docs.langchain.com/oss/python/migrate/langgraph-v1)

---

## Practice Exercise

**Challenge**: Build a complete expense approval agent:

1. **Tools**:
   - `submit_expense(amount, category, description)` - Always works
   - `approve_expense(expense_id, amount)` - Needs HITL if amount > $500
   - `get_expense_status(expense_id)` - Always works

2. **Features**:
   - Memory to track conversation
   - Structured output for expense reports
   - HITL approval for large expenses

3. **Test Scenarios**:
   - Submit small expense (auto-approved)
   - Submit large expense (needs approval)
   - Check status of submitted expenses

In [27]:
# Your solution here:

# Define Pydantic models
class ExpenseReport(BaseModel):
    """Structured expense report."""
    expense_id: str = Field(description="Unique expense ID")
    amount: float = Field(description="Amount in USD")
    category: str = Field(description="Expense category")
    description: str = Field(description="Expense description")
    status: str = Field(description="pending, approved, or rejected")

# TODO: Define tools
# @tool
# def submit_expense(...) -> str:
#     ...

# @tool  
# def approve_expense(...) -> str:
#     # Use interrupt() for amounts > $500
#     ...

# @tool
# def get_expense_status(...) -> str:
#     ...

# TODO: Create agent with memory and HITL

# TODO: Test scenarios

print("💡 Implement your expense approval agent above!")

💡 Implement your expense approval agent above!
